In [1]:
!pip install colabcode

Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this version.
INFO: pip is looking at multiple versions of colabcode to determine which version is compatible with other requirements. This could take a while.
  Using cached uvicorn-0.13.1-py3-none-any.whl.metadata (4.6 kB)
Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this versio

***#Set Up a Data Ingestion Pipeline***

In [ ]:
#install telethon first
!pip install telethon
!pip install python-dotenv



***Fetch Data from Telegram Channels***

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
import pandas as pd
from datetime import datetime

# Credentials
api_id = 25367099
api_hash = 'd7e39750bed65553fbceb37465d7689d'
phone = '+251966883822'

# Create client instance
client = TelegramClient('session_name', api_id, api_hash)

# Channel list
channels = [
    '@ZemenExpress',
    '@nevacomputer',
    '@ethio_brand_collection',
    '@meneshayeofficial',
    '@Leyueqa'
]

async def fetch_messages():
    await client.start(phone)

    all_data = []

    for ch in channels:
        try:
            entity = await client.get_entity(ch)

            messages = await client(GetHistoryRequest(
                peer=entity,
                limit=1000,  # Number of messages to fetch per channel
                offset_date=None,
                offset_id=0,
                max_id=0,
                min_id=0,
                add_offset=0,
                hash=0
            ))

            for msg in messages.messages:
                if hasattr(msg, 'message'):  # Check if message has text content
                    message_data = {
                        "channel": ch,
                        "date": msg.date,
                        "text": msg.message,
                        "views": getattr(msg, 'views', None),  # Some messages might not have views
                        "sender_id": getattr(msg, 'sender_id', None),
                        "message_id": msg.id,
                        "media_type": None
                    }

                    # Check for media
                    if hasattr(msg, 'media'):
                        if hasattr(msg.media, 'photo'):
                            message_data['media_type'] = 'photo'
                        elif hasattr(msg.media, 'document'):
                            message_data['media_type'] = 'document'

                    all_data.append(message_data)

        except Exception as e:
            print(f"Error in {ch}: {str(e)}")
            continue  # Continue with next channel if error occurs

    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv("telegram_scraped_data.csv", index=False)
        print(f"Successfully saved {len(df)} messages to telegram_scraped_data.csv")
        print(df.head())
    else:
        print("No messages were fetched. Check your channel list and connection.")

# Run the async task
await fetch_messages()

Successfully saved 500 messages to telegram_scraped_data.csv
         channel                      date  \
0  @ZemenExpress 2025-06-21 16:35:51+00:00   
1  @ZemenExpress 2025-06-21 16:35:51+00:00   
2  @ZemenExpress 2025-06-21 16:35:51+00:00   
3  @ZemenExpress 2025-06-21 16:35:51+00:00   
4  @ZemenExpress 2025-06-21 08:07:31+00:00   

                                                text   views      sender_id  \
0                                                      968.0 -1001307493052   
1                                                      968.0 -1001307493052   
2                                                      941.0 -1001307493052   
3  💥💥...................................💥💥\n\n📌Sa...   946.0 -1001307493052   
4                                                     1937.0 -1001307493052   

   message_id media_type  
0        6994      photo  
1        6993      photo  
2        6992      photo  
3        6991      photo  
4        6990      photo  


In [ ]:
import pandas as pd

# Load and display the CSV as a DataFrame
df = pd.read_csv('telegram_scraped_data.csv')

# Display the first 5 rows (or change to df.head(10) for 10 rows, etc.)
df.head(1000)


,channel,date,text,views,sender_id,message_id,media_type
0,@ZemenExpress,2025-06-21 16:35:51+00:00,NaN,953.0,-1001307493052,6994,photo
1,@ZemenExpress,2025-06-21 16:35:51+00:00,NaN,952.0,-1001307493052,6993,photo
2,@ZemenExpress,2025-06-21 16:35:51+00:00,NaN,926.0,-1001307493052,6992,photo
3,@ZemenExpress,2025-06-21 16:35:51+00:00,💥💥...................................💥💥\n\n📌Sa...,928.0,-1001307493052,6991,photo
4,@ZemenExpress,2025-06-21 08:07:31+00:00,NaN,1929.0,-1001307493052,6990,photo
...,...,...,...,...,...,...,...
495,@Leyueqa,2025-06-02 09:48:24+00:00,🔔🟢ፓኬጅ 27\n\n🔥የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ 2 ...,17220.0,-1001333826717,7510,photo
496,@Leyueqa,2025-06-02 07:56:47+00:00,🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤Soap dispensor የሳሙና መቀነሻ ተለጣፊ እቃ...,19369.0,-1001333826717,7509,document
497,@Leyueqa,2025-06-02 06:08:42+00:00,🔻🔻ሂና henna stencils designed for creating temp...,20074.0,-1001333826717,7508,document
498,@Leyueqa,2025-05-31 12:32:28+00:00,🤍🤍Spa Gel Socks\n\n🤍 ለእግር ልስላሴ \n🤍 ለሚሰነጣጠቅ እግር...,22783.0,-1001333826717,7507,document


Setup

In [ ]:
  !pip install etnltk



***Amharic text preprocessing with Amharic document***

In [ ]:
from etnltk import Amharic
import pandas as pd

# Load the CSV
df = pd.read_csv('telegram_scraped_data.csv')

# Use the correct column name (e.g., 'text' instead of 'Text')
df['Cleaned_Text'] = df['text'].apply(lambda x: str(Amharic(str(x))) if pd.notnull(x) else '')

# Show cleaned examples
df[['text', 'Cleaned_Text']].head(1000)


,text,Cleaned_Text
0,NaN,
1,NaN,
2,NaN,
3,💥💥...................................💥💥\n\n📌Sa...,"Amharic(""ዋጋ ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛመሰረትደፋርሞ..."
4,NaN,
...,...,...
495,🔔🟢ፓኬጅ 27\n\n🔥የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ 2 ...,"Amharic(""ፓኬጅ የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ ሊት..."
496,🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤Soap dispensor የሳሙና መቀነሻ ተለጣፊ እቃ...,"Amharic(""የሳሙና መቀነሻ ተለጣፊ እቃ ለሆስፒታል ለኪችንለፀጉርቤትለሆ..."
497,🔻🔻ሂና henna stencils designed for creating temp...,"Amharic(""ሂና ክፍያዎን ዕቃዉ እጅዎ ሲደርስከፈለጉበካሽአልያምበሞባይል..."
498,🤍🤍Spa Gel Socks\n\n🤍 ለእግር ልስላሴ \n🤍 ለሚሰነጣጠቅ እግር...,"Amharic(""ለእግር ልስላሴ ለሚሰነጣጠቅ እግር ድርቀትን የሚያለሰልስ የ..."


***Clean and Structure the Data***

In [ ]:
# 1. Install etnltk
!pip install -U etnltk

# 2. Imports
import pandas as pd
import re
from etnltk.lang.am import clean_amharic

# 3. Define custom cleaning functions
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_digits(text):
    return re.sub(r'\d+', '', text)

def remove_english_chars(text):
    return re.sub(r'[A-Za-z]', '', text)

def remove_ethiopic_punctuation(text):
    ethiopic_punct = "፡።፣፤፥፦፧፨"
    return re.sub(f"[{ethiopic_punct}]", '', text)

# 4. Load the CSV
df = pd.read_csv('telegram_scraped_data.csv')

# 5. Normalize column names
df.columns = df.columns.str.strip().str.lower()

# 6. Confirm the text column exists
if 'text' not in df.columns:
    raise KeyError("Expected a column named 'text' in the CSV.")

# 7. Define cleaning pipeline
custom_pipeline = [
    remove_emojis,
    remove_digits,
    remove_english_chars,
    remove_ethiopic_punctuation
]

# 8. Apply cleaning
df['cleaned_text'] = df['text'].apply(
    lambda x: clean_amharic(str(x), pipeline=custom_pipeline) if pd.notnull(x) else ''
)

# 9. Preview
df[['text', 'cleaned_text']].head(100)


,text,cleaned_text
0,NaN,
1,NaN,
2,NaN,
3,💥💥...................................💥💥\n\n📌Sa...,ዋጋ ብር ✅ ♦️ውስን ፍሬ ነው ያለው አድራሻ ♦️መገናኛመሰረትደፋርሞልሁለ...
4,NaN,
...,...,...
95,NaN,
96,NaN,
97,NaN,
98,💥💥...................................💥💥\n\n📌2 ...,የተረከዝ ሞረድ እና የፀጉር ማንሻ በ ቻርጅ የሚሰራ ዋጋ ብር ♦️ውስን ፍ...


Tokenization

In [ ]:
from etnltk import Amharic

# Helper function to process non-empty cleaned text
def extract_sentences(text):
    if pd.notnull(text) and text.strip() != '':
        return Amharic(text).sentences
    return []

# Apply to each row safely
df['sentences'] = df['cleaned_text'].apply(extract_sentences)

# Preview
df[['cleaned_text', 'sentences']].head(1000)


,cleaned_text,sentences
0,,[]
1,,[]
2,,[]
3,ዋጋ ብር ✅ ♦️ውስን ፍሬ ነው ያለው አድራሻ ♦️መገናኛመሰረትደፋርሞልሁለ...,"[Sentence(""ዋጋ ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛመሰረትደፋ..."
4,,[]
...,...,...
495,🟢ፓኬጅ የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ ሊትር የዉሃ ማፍ...,"[Sentence(""ፓኬጅ የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ ..."
496,የሳሙና መቀነሻ ተለጣፊ እቃ ለሆስፒታል ለኪችንለፀጉርቤትለሆቴልለመታጠቢያ ...,"[Sentence(""የሳሙና መቀነሻ ተለጣፊ እቃ ለሆስፒታል ለኪችንለፀጉርቤት..."
497,ሂና ክፍያዎን ዕቃዉ እጅዎ ሲደርስከፈለጉበካሽአልያምበሞባይልባንኪንግመፈፀም...,"[Sentence(""ሂና ክፍያዎን ዕቃዉ እጅዎ ሲደርስከፈለጉበካሽአልያምበሞባ..."
498,🤍🤍 🤍 ለእግር ልስላሴ 🤍 ለሚሰነጣጠቅ እግር 🤍 ድርቀትን የሚያለሰልስ 🤍...,"[Sentence(""ለእግር ልስላሴ ለሚሰነጣጠቅ እግር ድርቀትን የሚያለሰልስ..."


Tokenization

In [ ]:
from etnltk.tokenize.am import word_tokenize

# Tokenize each cleaned_text entry (if not empty)
df['tokens'] = df['cleaned_text'].apply(
    lambda x: word_tokenize(x) if pd.notnull(x) and x.strip() != '' else []
)

# Preview some tokens
df[['cleaned_text', 'tokens']].head(1000)


,cleaned_text,tokens
0,,[]
1,,[]
2,,[]
3,ዋጋ ብር ✅ ♦️ውስን ፍሬ ነው ያለው አድራሻ ♦️መገናኛመሰረትደፋርሞልሁለ...,"[ዋጋ, ብር, ውስን, ፍሬ, ነው, ያለው, አድራሻ, መገናኛመሰረትደፋርሞል..."
4,,[]
...,...,...
495,🟢ፓኬጅ የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ ሊትር የዉሃ ማፍ...,"[ፓኬጅ, የዉሃ, ማፍያ, ቦይለር, እና, የቡና, መፍጫ, አንድ, ላይ, ሊ..."
496,የሳሙና መቀነሻ ተለጣፊ እቃ ለሆስፒታል ለኪችንለፀጉርቤትለሆቴልለመታጠቢያ ...,"[የሳሙና, መቀነሻ, ተለጣፊ, እቃ, ለሆስፒታል, ለኪችንለፀጉርቤትለሆቴልለ..."
497,ሂና ክፍያዎን ዕቃዉ እጅዎ ሲደርስከፈለጉበካሽአልያምበሞባይልባንኪንግመፈፀም...,"[ሂና, ክፍያዎን, ዕቃዉ, እጅዎ, ሲደርስከፈለጉበካሽአልያምበሞባይልባንኪን..."
498,🤍🤍 🤍 ለእግር ልስላሴ 🤍 ለሚሰነጣጠቅ እግር 🤍 ድርቀትን የሚያለሰልስ 🤍...,"[ለእግር, ልስላሴ, ለሚሰነጣጠቅ, እግር, ድርቀትን, የሚያለሰልስ, የሚታ..."


In [ ]:
df['tokens'] = df['text'].apply(
    lambda x: word_tokenize(x) if pd.notnull(x) and x.strip() != '' else []
)
print(df[['text', 'tokens']].head(1000))

                                                  text  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3    💥💥...................................💥💥\n\n📌Sa...   
4                                                  NaN   
..                                                 ...   
495  🔔🟢ፓኬጅ 27\n\n🔥የዉሃ ማፍያ ቦይለር እና የቡና መፍጫ አንድ ላይ 2 ...   
496  🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤🔤Soap dispensor የሳሙና መቀነሻ ተለጣፊ እቃ...   
497  🔻🔻ሂና henna stencils designed for creating temp...   
498  🤍🤍Spa Gel Socks\n\n🤍 ለእግር ልስላሴ \n🤍 ለሚሰነጣጠቅ እግር...   
499                                                NaN   

                                                tokens  
0                                                   []  
1                                                   []  
2                                                   []  
3    [ዋጋ, ብር, ውስን, ፍሬ, ነው, ያለው, አድራሻ, መገናኛ, መሰረት, ደ...  
4                 

In [ ]:
# Example: Clean raw text (remove unwanted characters, normalize, etc.)
df['cleaned_text'] = df['text'].str.replace(r'[^\u1200-\u137F\s]', '', regex=True)  # Keep Ge'ez script and spaces

In [ ]:
import pandas as pd
from etnltk.tokenize.am import word_tokenize

# Create subset (e.g., first 100 rows)
subset_df = df.head(100)  # Adjust as needed

# Save tokens to a text file for manual annotation
with open('amharic_tokens_for_labeling.txt', 'w', encoding='utf-8') as f:
    for idx, row in subset_df.iterrows():
        if row['tokens']:  # Skip empty token lists
            f.write(f"# text: {row['text']}\n")  # Use correct column name
            for token in row['tokens']:
                f.write(f"{token}\t_\t_\n")  # Placeholder for POS, NER
            f.write("\n")

In [ ]:
# Add placeholder tags (replace with actual annotations if available)
subset_df['ner_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['O'] * len(tokens) if tokens else []
)
subset_df['pos_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['_'] * len(tokens) if tokens else []
)

# Save to CoNLL
def save_to_conll(df, output_file, token_col='tokens', ner_col='ner_tags', pos_col='pos_tags'):
    with open(output_file, 'w', encoding='utf-8') as f:
        for idx, row in df.iterrows():
            if not row[token_col]:  # Skip empty token lists
                continue
            f.write(f"# text: {row['text']}\n")  # Use correct column name
            tokens = row[token_col]
            ner_tags = row[ner_col] if ner_col in df.columns else ['O'] * len(tokens)
            pos_tags = row[pos_col] if pos_col in df.columns else ['_'] * len(tokens)

            for token, pos, ner in zip(tokens, pos_tags, ner_tags):
                f.write(f"{token}\t{pos}\t{ner}\n")
            f.write("\n")

save_to_conll(subset_df, 'amharic_ner.conll')

/tmp/ipython-input-90-981128443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['ner_tags'] = subset_df['tokens'].apply(
/tmp/ipython-input-90-981128443.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['pos_tags'] = subset_df['tokens'].apply(


In [ ]:
import pandas as pd
from etnltk.tokenize.am import word_tokenize

# Assuming df is your DataFrame
subset_df = df.head(100)
subset_df['ner_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['O'] * len(tokens) if tokens else []
)
subset_df['pos_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['_'] * len(tokens) if tokens else []
)

def save_to_conll(df, output_file, token_col='tokens', ner_col='ner_tags', pos_col='pos_tags'):
    with open(output_file, 'w', encoding='utf-8') as f:
        for idx, row in df.iterrows():
            if not row[token_col]:  # Skip empty token lists
                continue
            f.write(f"# text: {row['text']}\n")  # Assuming 'text' column
            tokens = row[token_col]
            ner_tags = row[ner_col] if ner_col in df.columns else ['O'] * len(tokens)
            pos_tags = row[pos_col] if pos_col in df.columns else ['_'] * len(tokens)

            for token, pos, ner in zip(tokens, pos_tags, ner_tags):
                f.write(f"{token}\t{pos}\t{ner}\n")
            f.write("\n")

save_to_conll(subset_df, 'amharic_ner.conll')


/tmp/ipython-input-110-3217002178.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['ner_tags'] = subset_df['tokens'].apply(
/tmp/ipython-input-110-3217002178.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['pos_tags'] = subset_df['tokens'].apply(


In [ ]:
print(subset_df.columns)
print(subset_df[['tokens', 'ner_tags']].head(1000))

Index(['channel', 'date', 'text', 'views', 'sender_id', 'message_id',
       'media_type', 'tokens', 'cleaned_text', 'ner_tags', 'pos_tags'],
      dtype='object')
                                               tokens  \
0                                                  []   
1                                                  []   
2                                                  []   
3   [ዋጋ, ብር, ውስን, ፍሬ, ነው, ያለው, አድራሻ, መገናኛ, መሰረት, ደ...   
4                                                  []   
..                                                ...   
95                                                 []   
96                                                 []   
97                                                 []   
98  [የተረከዝ, ሞረድ, እና, የፀጉር, ማንሻ, በ, ቻርጅ, የሚሰራ, ዋጋ, ...   
99  [የተረከዝ, ሞረድ, እና, የፀጉር, ማንሻ, በ, ቻርጅ, የሚሰራ, ዋጋ, ...   

                                             ner_tags  
0                                                  []  
1                                      

In [ ]:
# Check for length mismatches
for idx, row in subset_df.iterrows():
    if row['tokens'] and len(row['tokens']) != len(row['ner_tags']):
        print(f"Row {idx}: Token-NER mismatch. Tokens: {len(row['tokens'])}, NER tags: {len(row['ner_tags'])}")
    if row['tokens'] and len(row['tokens']) != len(row['pos_tags']):
        print(f"Row {idx}: Token-POS mismatch. Tokens: {len(row['tokens'])}, POS tags: {len(row['pos_tags'])}")

In [ ]:
subset_df['ner_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['O'] * len(tokens) if tokens else []
)
subset_df['pos_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['_'] * len(tokens) if tokens else []
)

/tmp/ipython-input-104-383550501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['ner_tags'] = subset_df['tokens'].apply(
/tmp/ipython-input-104-383550501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['pos_tags'] = subset_df['tokens'].apply(


In [ ]:
save_to_conll(subset_df, 'amharic_ner.conll')

In [ ]:
with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
    print("\n".join(f.readlines()[:1000]))

# text: 💥💥...................................💥💥



📌Saachi Electric Kettle



👍Borosilicate Glass Body

👍Overheat protection

👍Automatic switch off

👍2200w



ዋጋ፦  💲🏷 2700  ብር ✅



♦️ውስን ፍሬ ነው ያለው🔥🔥🔥



🏢 አድራሻ👉



📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06





     💧💧💧💧





    📲 0902660722

    📲 0928460606 



🔖

💬በTelegram ለማዘዝ ⤵️ ይጠቀሙ🔽



@zemencallcenter 



@zemenexpressadmin



ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን⤵️

https://telegram.me/zemenexpress



💥💥...................................💥💥

ዋጋ	_	O

ብር	_	O

ውስን	_	O

ፍሬ	_	O

ነው	_	O

ያለው	_	O

አድራሻ	_	O

መገናኛ	_	O

መሰረት	_	O

ደፋር	_	O

ሞል	_	O

ሁለተኛ	_	O

ፎቅ	_	O

ቢሮ	_	O

ቁ	_	O

በ	_	O

ለማዘዝ	_	O

ይጠቀሙ	_	O

ለተጨማሪ	_	O

ማብራሪያ	_	O

የቴሌግራም	_	O

ገፃችን	_	O



# text: 💥💥...................................💥💥



3pcs Bottle Stopper 



👍በማንኛውም ጠርሙስ ጫፍ የሚገጠም 

👍ለዘይት እና መሰል ነገሮች መቀነሻ የሚሆን

👍በአግባቡ እየመጠንን ለመጠቀም ተመራጭ



ዋጋ፦  💲🏷  3ፍሬ➡️ 400 ብር



♦️ውስን ፍሬ ነው ያለው 🔥🔥🔥



🏢 አድራሻ👉



📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06





     💧💧💧💧





    📲 0902660722

    📲 0

In [ ]:
import pandas as pd
from etnltk.tokenize.am import word_tokenize
import re

# Verify DataFrame exists
if 'df' not in globals():
    raise NameError("DataFrame 'df' is not defined. Load or create it first.")
if 'text' not in df.columns:
    raise KeyError("Column 'text' not found. Available columns: " + str(df.columns.tolist()))

# Create a copy of subset to avoid SettingWithCopyWarning
subset_df = df.head(100).copy()

# Tokenize if not already done
if 'tokens' not in subset_df.columns:
    subset_df['tokens'] = subset_df['text'].apply(
        lambda x: word_tokenize(x) if pd.notnull(x) and x.strip() != '' else []
    )

# Filter out rows with non-Amharic or problematic text
def is_valid_amharic_text(text):
    if pd.isna(text) or not text.strip():
        return False
    # Keep text with at least some Amharic characters (Ge'ez script: U+1200–U+137F)
    return bool(re.search(r'[\u1200-\u137F]', text))

subset_df = subset_df[subset_df['text'].apply(is_valid_amharic_text)]

# Add placeholder tags
subset_df.loc[:, 'ner_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['O'] * len(tokens) if tokens else []
)
subset_df.loc[:, 'pos_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['_'] * len(tokens) if tokens else []
)

# Debug: Check for empty tokens
empty_token_rows = subset_df[subset_df['tokens'].apply(len) == 0]
if not empty_token_rows.empty:
    print(f"Warning: {len(empty_token_rows)} rows have empty tokens:")
    print(empty_token_rows[['text', 'tokens']].head())

# Save to CoNLL
def save_to_conll(df, output_file, token_col='tokens', ner_col='ner_tags', pos_col='pos_tags'):
    with open(output_file, 'w', encoding='utf-8') as f:
        sentence_count = 0
        for idx, row in df.iterrows():
            if not row[token_col]:
                continue
            # Only include text with Amharic content in comments
            if not is_valid_amharic_text(row['text']):
                continue
            f.write(f"# text: {row['text']}\n")
            tokens = row[token_col]
            ner_tags = row[ner_col] if ner_col in df.columns else ['O'] * len(tokens)
            pos_tags = row[pos_col] if pos_col in df.columns else ['_'] * len(tokens)

            if len(tokens) != len(ner_tags) or len(tokens) != len(pos_tags):
                print(f"Row {idx}: Mismatch. Tokens: {len(tokens)}, NER: {len(ner_tags)}, POS: {len(pos_tags)}")
                continue

            for token, pos, ner in zip(tokens, pos_tags, ner_tags):
                # Skip tokens that are purely emojis or non-text
                if not re.match(r'^[\u1200-\u137F0-9a-zA-Z]+$', token):
                    continue
                f.write(f"{token}\t{pos}\t{ner}\n")
            f.write("\n")
            sentence_count += 1
        print(f"Wrote {sentence_count} sentences to {output_file}")

save_to_conll(subset_df, 'amharic_ner.conll')

# Validate
from conllu import parse
try:
    with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
        data = f.read()
    sentences = parse(data)
    print(f"Parsed {len(sentences)} sentences")
except Exception as e:
    print(f"Parsing failed: {e}")
    with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
        print("\nFirst 20 lines of file:")
        print("\n".join(f.readlines()[:20]))

Wrote 40 sentences to amharic_ner.conll
Parsing failed: Invalid line format, line must contain either tabs or two spaces.

First 20 lines of file:
# text: 💥💥...................................💥💥



📌Saachi Electric Kettle



👍Borosilicate Glass Body

👍Overheat protection

👍Automatic switch off

👍2200w



ዋጋ፦  💲🏷 2700  ብር ✅



♦️ውስን ፍሬ ነው ያለው🔥🔥🔥



🏢 አድራሻ👉



📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06





     💧💧💧💧





In [ ]:
def rule_based_ner(tokens):
    ner_tags = []
    for i, token in enumerate(tokens):
        if token in {'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል'}:
            ner_tags.append('B-LOC' if i == 0 or tokens[i-1] not in {'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል'} else 'I-LOC')
        elif token.startswith('@'):
            ner_tags.append('B-ORG')
        elif token == 'ብር':
            ner_tags.append('I-PRICE' if i > 0 and tokens[i-1].isdigit() else 'O')
        elif token.isdigit():
            ner_tags.append('B-PRICE')
        else:
            ner_tags.append('O')
    return ner_tags

subset_df.loc[:, 'ner_tags'] = subset_df['tokens'].apply(rule_based_ner)
save_to_conll(subset_df, 'amharic_ner.conll')

Wrote 40 sentences to amharic_ner.conll


In [ ]:
import pandas as pd
from etnltk.tokenize.am import word_tokenize
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import re
from conllu import parse

# Verify DataFrame exists
if 'df' not in globals():
    raise NameError("DataFrame 'df' is not defined. Load or create it first.")
if 'text' not in df.columns:
    raise KeyError("Column 'text' not found. Available columns: " + str(df.columns.tolist()))

# Create a copy of subset to avoid SettingWithCopyWarning
subset_df = df.head(100).copy()

# Tokenize if not already done
if 'tokens' not in subset_df.columns:
    subset_df['tokens'] = subset_df['text'].apply(
        lambda x: word_tokenize(x) if pd.notnull(x) and x.strip() != '' else []
    )

# Filter out rows with non-Amharic or problematic text
def is_valid_amharic_text(text):
    if pd.isna(text) or not text.strip():
        return False
    return bool(re.search(r'[\u1200-\u137F]', text))  # Ge'ez script

subset_df = subset_df[subset_df['text'].apply(is_valid_amharic_text)]

# Debug: Check for empty tokens
empty_token_rows = subset_df[subset_df['tokens'].apply(len) == 0]
if not empty_token_rows.empty:
    print(f"Warning: {len(empty_token_rows)} rows have empty tokens:")
    print(empty_token_rows[['text', 'tokens']].head())

# Initialize NER pipeline
try:
    tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
except OSError as e:
    print(f"Error loading model: {e}")
    print("Ensure you have internet access and the correct model identifier.")
    raise

# Predict NER tags with token alignment
def predict_ner(tokens):
    if not tokens:
        return []
    text = " ".join(tokens)
    try:
        predictions = ner_pipeline(text)
    except Exception as e:
        print(f"NER pipeline failed for text: {text}\nError: {e}")
        return ['O'] * len(tokens)

    # Align predictions with tokens
    ner_tags = ['O'] * len(tokens)
    token_idx = 0
    current_offset = 0
    for pred in predictions:
        entity = pred['entity_group']
        start = pred['start']
        end = pred['end']
        # Find which token(s) the entity spans
        token_text = "".join(tokens)  # Concatenate without spaces for offset calculation
        while token_idx < len(tokens):
            token = tokens[token_idx]
            token_start = current_offset
            token_end = current_offset + len(token)
            if start >= token_end:
                current_offset += len(token) + 1  # +1 for space
                token_idx += 1
                continue
            if end <= token_start:
                break
            # Assign entity to the token (simplified: assign to first token of entity)
            ner_tags[token_idx] = entity
            break
    return ner_tags

# Apply NER predictions
subset_df.loc[:, 'ner_tags'] = subset_df['tokens'].apply(predict_ner)
subset_df.loc[:, 'pos_tags'] = subset_df['tokens'].apply(
    lambda tokens: ['_'] * len(tokens) if tokens else []
)

# Save to CoNLL
def save_to_conll(df, output_file, token_col='tokens', ner_col='ner_tags', pos_col='pos_tags'):
    with open(output_file, 'w', encoding='utf-8') as f:
        sentence_count = 0
        for idx, row in df.iterrows():
            if not row[token_col]:
                continue
            if not is_valid_amharic_text(row['text']):
                continue
            f.write(f"# text: {row['text']}\n")
            tokens = row[token_col]
            ner_tags = row[ner_col] if ner_col in df.columns else ['O'] * len(tokens)
            pos_tags = row[pos_col] if pos_col in df.columns else ['_'] * len(tokens)

            if len(tokens) != len(ner_tags) or len(tokens) != len(pos_tags):
                print(f"Row {idx}: Mismatch. Tokens: {len(tokens)}, NER: {len(ner_tags)}, POS: {len(pos_tags)}")
                continue

            for token, pos, ner in zip(tokens, pos_tags, ner_tags):
                if not re.match(r'^[\u1200-\u137F0-9a-zA-Z]+$', token):
                    continue
                f.write(f"{token}\t{pos}\t{ner}\n")
            f.write("\n")
            sentence_count += 1
        print(f"Wrote {sentence_count} sentences to {output_file}")

save_to_conll(subset_df, 'amharic_ner.conll')

# Validate
try:
    with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
        data = f.read()
    sentences = parse(data)
    print(f"Parsed {len(sentences)} sentences")
except Exception as e:
    print(f"Parsing failed: {e}")
    with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
        print("\nFirst 20 lines of file:")
        print("\n".join(f.readlines()[:100]))

Device set to use cpu


Wrote 40 sentences to amharic_ner.conll
Parsing failed: Invalid line format, line must contain either tabs or two spaces.

First 20 lines of file:
# text: 💥💥...................................💥💥



📌Saachi Electric Kettle



👍Borosilicate Glass Body

👍Overheat protection

👍Automatic switch off

👍2200w



ዋጋ፦  💲🏷 2700  ብር ✅



♦️ውስን ፍሬ ነው ያለው🔥🔥🔥



🏢 አድራሻ👉



📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06





     💧💧💧💧





    📲 0902660722

    📲 0928460606 



🔖

💬በTelegram ለማዘዝ ⤵️ ይጠቀሙ🔽



@zemencallcenter 



@zemenexpressadmin



ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን⤵️

https://telegram.me/zemenexpress



💥💥...................................💥💥

ዋጋ	_	O

ብር	_	O

ውስን	_	O

ፍሬ	_	O

ነው	_	O

ያለው	_	O

አድራሻ	_	O

መገናኛ	_	O

መሰረት	_	O

ደፋር	_	O

ሞል	_	O

ሁለተኛ	_	O

ፎቅ	_	O

ቢሮ	_	O

ቁ	_	O

በ	_	O

ለማዘዝ	_	O

ይጠቀሙ	_	O

ለተጨማሪ	_	O

ማብራሪያ	_	O

የቴሌግራም	_	O

ገፃችን	_	O



# text: 💥💥...................................💥💥



3pcs Bottle Stopper 



👍በማንኛውም ጠርሙስ ጫፍ የሚገጠም 

👍ለዘይት እና መሰል ነገሮች መቀነሻ የሚሆን

👍በአግባቡ እየመጠንን ለመጠቀም ተመራጭ





In [ ]:
def rule_based_ner(tokens):
    ner_tags = []
    for i, token in enumerate(tokens):
        if token in {'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል'}:
            ner_tags.append('B-LOC' if i == 0 or tokens[i-1] not in {'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል'} else 'I-LOC')
        elif token.startswith('@'):
            ner_tags.append('B-ORG')
        elif token == 'ብር':
            ner_tags.append('I-PRICE' if i > 0 and tokens[i-1].isdigit() else 'O')
        elif token.isdigit():
            ner_tags.append('B-PRICE')
        else:
            ner_tags.append('O')
    return ner_tags

subset_df.loc[:, 'ner_tags'] = subset_df['tokens'].apply(rule_based_ner)
save_to_conll(subset_df, 'amharic_ner.conll')

Wrote 40 sentences to amharic_ner.conll


In [ ]:
with open('amharic_ner.conll', 'r', encoding='utf-8') as f:
    print("\n".join(f.readlines()[:20]))

# text: 💥💥...................................💥💥



📌Saachi Electric Kettle



👍Borosilicate Glass Body

👍Overheat protection

👍Automatic switch off

👍2200w



ዋጋ፦  💲🏷 2700  ብር ✅



♦️ውስን ፍሬ ነው ያለው🔥🔥🔥



🏢 አድራሻ👉



📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06





     💧💧💧💧



